In [ ]:
import keras
from keras.optimizers.legacy import Adam as LegacyAdam
import numpy as np
from src.utils.preprocessing import load_split_data
import json
from src.utils.data_io import save_data

In [ ]:
with open("../data/dataset-info-json/subject_to_indices.json", "r") as f:
    subject_to_indices = json.load(f)

subject_to_indices = {int(k): v for k, v in subject_to_indices.items()}

In [ ]:
for test_subject_id in subject_to_indices.keys():
    model = keras.models.load_model(f"../models/full_loso/majority_label/model_{test_subject_id}.keras", compile=False)
    optimizer = LegacyAdam(learning_rate=1e-3)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])
    _, _, test_data, test_labels = load_split_data(test_subject_id, subject_to_indices)
    predictions = []
    for window in test_data:
        window_reshaped = window.reshape(1,20,6)
        prediction = model.predict(window_reshaped)
        predictions.append(prediction)
    predictions = np.array(predictions)
    predictions = predictions.squeeze(axis=1)
    save_data(predictions, "../data/cnn_predictions/non-majority/test", f"prediction_{test_subject_id}")